In [1]:
import numpy as np
import pytest

import sys
import os

# Add the src directory to the path
project_root = "/Users/yjinstat/Desktop/Research/collaborations/cp-diagnosis/StratCP"
src_path = os.path.join(project_root, "src")
sys.path.insert(0, src_path)

# from stratcp.selection.single import get_reference_sel_single, get_sel_single
# from stratcp.selection.multiple import get_reference_sel_multiple, get_sel_multiple
# from stratcp.conformal import conformal
# from stratcp.conformal.scores import compute_score_aps

In [79]:

n, m = 1000, 500
K = 10
alpha = 0.3

fdp = []
for seed in range(1000):
    np.random.seed(seed)
    # Generate synthetic data
    cal_scores = np.random.dirichlet(np.ones(K), size=n)
    cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])  
    cal_confs = 1 * (cal_labels == np.argmax(cal_scores, axis=1))
    cal_eligs = np.ones(n)

    val_scores = np.random.dirichlet(np.ones(K), size=m)
    val_eligs = np.ones(m)
    val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  

    sel_idx, unsel_idx, hat_tau = get_sel_single(
    cal_scores[:,0], cal_eligs, cal_confs, val_scores[:,0], val_eligs, alpha)

    fdp.append(np.mean(val_labels[sel_idx] != np.argmax(val_scores[sel_idx], axis=1)) if len(sel_idx) > 0 else 0)

print(np.mean(fdp))
print(fdp)

0.2193500900099216
[np.float64(0.6363636363636364), 0, 0, np.float64(0.0), np.float64(0.6666666666666666), 0, np.float64(0.5454545454545454), 0, np.float64(1.0), 0, 0, 0, 0, 0, 0, 0, np.float64(0.5714285714285714), 0, 0, np.float64(0.6666666666666666), 0, np.float64(0.75), 0, np.float64(0.68), np.float64(0.3333333333333333), np.float64(0.6666666666666666), 0, np.float64(0.5333333333333333), 0, np.float64(0.6), 0, 0, 0, np.float64(0.6818181818181818), 0, 0, 0, 0, 0, 0, 0, 0, np.float64(0.6), np.float64(0.4166666666666667), np.float64(0.5), 0, 0, np.float64(0.55), np.float64(0.42857142857142855), np.float64(0.5555555555555556), np.float64(1.0), np.float64(0.8), 0, 0, np.float64(0.42857142857142855), np.float64(0.25), np.float64(0.25), 0, 0, 0, np.float64(1.0), 0, np.float64(0.4), 0, np.float64(0.6470588235294118), 0, 0, np.float64(0.6363636363636364), np.float64(0.38461538461538464), 0, 0, 0, np.float64(0.0), 0, 0, 0, np.float64(0.5), np.float64(0.2857142857142857), np.float64(0.6), 0, n

In [44]:


np.random.seed(0)
n, m = 1000, 500
K = 10

# Generate synthetic data
cal_scores = np.random.dirichlet(np.ones(K), size=n)
cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])  
cal_errs = 1 * (cal_labels == 0)
cal_eligs = np.ones(n)

val_scores = np.random.dirichlet(np.ones(K), size=m)
val_eligs = np.ones(m)
val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  

np.mean(np.argmax(val_scores, axis=1) != val_labels)

np.float64(0.716)

In [ ]:
alpha = 0.1
sel_idx, unsel_idx, hat_tau = get_sel_single(
    cal_scores[:,0], cal_eligs, cal_errs, val_scores[:,0], val_eligs, alpha
)

val_imputed_labels = np.zeros((m,K))
val_imputed_labels[:, 0] = 1


In [49]:

ref_mats = get_reference_sel_single(
    unsel_idx, cal_errs, cal_eligs, cal_scores[:,0], val_eligs, val_scores[:,0], val_imputed_labels, alpha
) 

for k in range(K):
    ref_mats[k] = ref_mats[k][unsel_idx,:]

In [50]:

calib_scores_aps, test_scores_aps = compute_score_aps(
    cal_smx = cal_scores,
    val_smx = val_scores,
    cal_labels = cal_labels,
    val_labels = None
) 

aps_set = conformal(
    calib_scores_aps, test_scores_aps[unsel_idx,:], 
    cal_labels, 
    alpha, nonempty=False,
    if_in_ref=ref_mats)


In [52]:
np.mean([aps_set[i,val_labels[unsel_idx[i]]] for i in range(len(unsel_idx))])

np.float64(0.907070707070707)

# Test multiple 

In [ ]:
np.random.seed(0)
n, m = 1000, 500
K = 10


# Generate synthetic data
cal_scores = np.random.dirichlet(np.ones(K), size=n)
cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])  
cal_argmax = np.argmax(cal_scores, axis=1)
cal_eligs = np.zeros((n, K))
for k in range(K):
    cal_eligs[:, k] = 1 * (cal_argmax == k)
cal_confs = np.zeros((n, K))
for k in range(K):
    cal_confs[:, k] = 1 * (cal_labels == k) 

val_scores = np.random.dirichlet(np.ones(K), size=m)
val_eligs = np.zeros((m, K))
for k in range(K):
    val_eligs[:, k] = 1 * (np.argmax(val_scores, axis=1) == k)
val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  
 

In [72]:

alpha = 0.4

all_sel, tau_list = get_sel_multiple(
    cal_scores, cal_eligs, cal_confs, val_scores, val_eligs, alpha
)

cal_confs = np.zeros((n, K))
for k in range(K):
    cal_confs[:, k] = 1 * (cal_labels == k)
val_imputed_confs = []
for k in range(K):
    conf_k = np.zeros((m, K))
    conf_k[:, k] = 1 
    val_imputed_confs.append(conf_k) 

ref_mats = get_reference_sel_multiple(
    all_sel[K], cal_confs, cal_eligs, cal_scores, val_eligs, val_scores, val_imputed_confs, alpha  
)


In [74]:
for k in range(K):
    ref_mats[k] = ref_mats[k][all_sel[K],:]

In [77]:
calib_scores_aps, test_scores_aps = compute_score_aps(
    cal_smx = cal_scores,
    val_smx = val_scores,
    cal_labels = cal_labels,
    val_labels = None
) 

aps_set = conformal(
    calib_scores_aps, test_scores_aps[all_sel[K],:], 
    cal_labels, 
    alpha, nonempty=False,
    if_in_ref=ref_mats)

np.mean([aps_set[i,val_labels[all_sel[K][i]]] for i in range(len(all_sel[K]))])

np.float64(0.5607064017660044)

In [ ]:
N = 1000
fdp = np.zeros((N, K))
eligs = np.zeros((N, K))
covs = []
for i in range(N):
    np.random.seed(i)
    # Generate synthetic data
    cal_scores = np.random.dirichlet(np.ones(K), size=n)
    cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])  
    cal_argmax = np.argmax(cal_scores, axis=1)
    cal_eligs = np.zeros((n, K))
    for k in range(K):
        cal_eligs[:, k] = 1 * (cal_argmax == k)
    cal_confs = np.zeros((n, K))
    for k in range(K):
        cal_confs[:, k] = 1 * (cal_labels == k) 

    val_scores = np.random.dirichlet(np.ones(K), size=m)
    val_eligs = np.zeros((m, K))
    for k in range(K):
        val_eligs[:, k] = 1 * (np.argmax(val_scores, axis=1) == k)
    val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  

    # conformal selection 
    all_sel, tau_list = get_sel_multiple(
    cal_scores, cal_eligs, cal_confs, val_scores, val_eligs, alpha
    )

    for k in range(K):
        sel_k = (all_sel[k]).astype(int)
        eligs[i, k] = np.sum(np.argmax(val_scores, axis=1)[sel_k] != k)
        fdp[i, k] = np.mean(val_labels[sel_k] == k) if len(sel_k)>0 else 0.


    # JOMI 
    cal_confs = np.zeros((n, K))
    for k in range(K):
        cal_confs[:, k] = 1 * (cal_labels == k)
    val_imputed_confs = []
    for k in range(K):
        conf_k = np.zeros((m, K))
        conf_k[:, k] = 1 
        val_imputed_confs.append(conf_k) 

    ref_mats = get_reference_sel_multiple(
        all_sel[K], cal_confs, cal_eligs, cal_scores, val_eligs, val_scores, val_imputed_confs, alpha  
    )

    for k in range(K):
        ref_mats[k] = ref_mats[k][all_sel[K],:]

    calib_scores_aps, test_scores_aps = compute_score_aps(
        cal_smx = cal_scores,
        val_smx = val_scores,
        cal_labels = cal_labels,
        val_labels = None
    ) 
    aps_set = conformal(
        calib_scores_aps, test_scores_aps[all_sel[K],:], 
        cal_labels, 
        alpha, nonempty=False,
        if_in_ref=ref_mats)

    covs.append(np.mean([aps_set[i,val_labels[all_sel[K][i]]] for i in range(len(all_sel[K]))]))



In [ ]:
print(np.mean(fdp, axis=0))
print(np.mean(eligs, axis=0)) 
print(np.mean(covs))

[0.12730585 0.11515928 0.11854706 0.13743633 0.12556882 0.12107369
 0.12852374 0.11788325 0.1218671  0.12333562]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.6997838854314393


In [70]:
print(np.mean(fdp, axis=0))
print(alpha)

[0.27271334 0.27209261 0.25553102 0.26274259 0.26196557 0.27240502
 0.26394708 0.23950495 0.24574231 0.2552369 ]
0.4


# Test unified API

In [2]:
from stratcp.stratified import StratifiedCP

In [ ]:
n = 1000
m = 500
K = 10
alpha = 0.3

cal_scores = np.random.dirichlet(np.ones(K), size=n)
cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])  
cal_confs = 1 * (cal_labels == np.argmax(cal_scores, axis=1))
cal_eligs = np.ones(n)

val_scores = np.random.dirichlet(np.ones(K), size=m)
val_eligs = np.ones(m)
val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  

sel_idx, unsel_idx, hat_tau = get_sel_single(np.max(cal_scores, axis=1), cal_eligs, cal_confs, np.max(val_scores, axis=1), val_eligs, alpha)

val_imputed_labels = np.zeros((m,K))
for k in range(K):
    val_imputed_labels[:, k] = 1 * (np.argmax(val_scores, axis=1) == k) 

ref_mats = get_reference_sel_single(
    unsel_idx, cal_confs, cal_eligs, np.max(cal_scores, axis=1), 
    val_eligs, np.max(val_scores, axis=1), val_imputed_labels, alpha
) 

for k in range(K):
    ref_mats[k] = ref_mats[k][unsel_idx,:]

calib_scores_aps, test_scores_aps = compute_score_aps(
    cal_smx = cal_scores,
    val_smx = val_scores,
    cal_labels = cal_labels,
    val_labels = None
) 

aps_set = conformal(
    calib_scores_aps, test_scores_aps[unsel_idx,:], 
    cal_labels, 
    alpha, nonempty=False, 
    if_in_ref=ref_mats, rand=False)


scp = StratifiedCP(eligibility="overall", alpha_sel=0.3, alpha_cp=0.3, nonempty=False, rand=False)
scp_res = scp.fit_predict(cal_scores, cal_labels, val_scores)

print("Raw selected:\n", sel_idx)
print("API selected:\n", scp_res['selected_idx'])
print("Prediction sets no-match:", np.sum(aps_set != scp_res['prediction_sets']))

Raw selected:
 [162 179 180 188 244 383 425 479 498]
API selected:
 [162 179 180 188 244 383 425 479 498]
Prediction sets no-match: 0


In [8]:
np.where(aps_set != scp_res['prediction_sets'])

(array([ 15,  48,  54, 107, 142, 152, 264, 316, 325, 344, 372, 390, 399,
        400, 418, 444]),
 array([3, 1, 8, 7, 3, 5, 4, 6, 7, 4, 7, 7, 2, 5, 4, 6]))

### selection by class

In [ ]:
n = 1000
m = 500
K = 10
alpha = 0.3

cal_scores = np.random.dirichlet(np.ones(K), size=n)
cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])  
cal_argmax = np.argmax(cal_scores, axis=1)
cal_eligs = np.zeros((n, K))
for k in range(K):
    cal_eligs[:, k] = 1 * (cal_argmax == k)
cal_confs = np.zeros((n, K))
for k in range(K):
    cal_confs[:, k] = 1 * (cal_labels == k) 

val_scores = np.random.dirichlet(np.ones(K), size=m)
val_eligs = np.zeros((m, K))
for k in range(K):
    val_eligs[:, k] = 1 * (np.argmax(val_scores, axis=1) == k)
val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  

# conformal selection 
all_sel, tau_list = get_sel_multiple(
    cal_scores, cal_eligs, cal_confs, val_scores, val_eligs, alpha
)


# JOMI 
cal_confs = np.zeros((n, K))
for k in range(K):
    cal_confs[:, k] = 1 * (cal_labels == k)
val_imputed_confs = []
for k in range(K):
    conf_k = np.zeros((m, K))
    conf_k[:, k] = 1 
    val_imputed_confs.append(conf_k) 

ref_mats = get_reference_sel_multiple(
    all_sel[K], cal_confs, cal_eligs, cal_scores, val_eligs, val_scores, val_imputed_confs, alpha  
)

for k in range(K):
    ref_mats[k] = ref_mats[k][all_sel[K],:] 

calib_scores_aps, test_scores_aps = compute_score_aps(
    cal_smx = cal_scores,
    val_smx = val_scores,
    cal_labels = cal_labels,
    val_labels = None
) 
aps_set = conformal(
    calib_scores_aps, test_scores_aps[all_sel[K],:], 
    cal_labels, 
    alpha, nonempty=False,
    if_in_ref=ref_mats, rand=False)


In [2]:
from stratcp.stratified import StratifiedCP

n, m, K = 1000, 500, 5
alpha = 0.2

np.random.seed(2)
cal_probs = np.random.dirichlet(np.ones(K)*0.5, size=n)
cal_labels = np.array([np.random.choice(K, p=cal_probs[i,:]) for i in range(n)])   
val_probs = np.random.dirichlet(np.ones(K)*0.5, size=m) 
val_labels = np.array([np.random.choice(K, p=val_probs[i]) for i in range(m)])  

scp = StratifiedCP(alpha_sel=0.3, alpha_cp=0.3)
results = scp.fit_predict(cal_probs, cal_labels, val_probs, val_labels)

# Access results
print(f"Selected: {len(results['selected_idx'])} high-confidence predictions") 
print(f"Avg set size (unselected): {results['set_sizes'].mean():.2f}")

# Print detailed summary
print(scp.summary())

Selected: 137 high-confidence predictions
Avg set size (unselected): 1.92
Stratified Conformal Prediction Summary (Overall Eligibility)
Score function: aps
Selection FDR level (α_sel): 0.3
CP miscoverage level (α_cp): 0.3

Selection Results:
  Total samples: 500
  Selected (high confidence): 137 (27.4%)
  Unselected (low confidence): 363 (72.6%)
  Selection threshold: 0.631

Prediction Set Sizes (unselected):
  Mean: 1.92, Median: 2


In [3]:
import stratcp as scp

model = scp.StratifiedCP(alpha_sel=0.3, alpha_cp=0.3, eligibility='per_class')
results = model.fit_predict(cal_probs, cal_labels, val_probs, val_labels)

# Print detailed summary
print(model.summary())

Stratified Conformal Prediction Summary (Per-Class Eligibility)
Score function: aps
Selection FDR level (α_sel): 0.3
CP miscoverage level (α_cp): 0.3
Number of classes: 5

Selection Results by Class:
  Total test samples: 500

  Class 0: 69 selected (13.8%)
    Threshold: 0.511
  Class 1: 37 selected (7.4%)
    Threshold: 0.591
  Class 2: 27 selected (5.4%)
    Threshold: 0.621
  Class 3: 56 selected (11.2%)
    Threshold: 0.518
  Class 4: 0 selected (0.0%)
    Threshold: 1.000

  Unselected (not selected by any class): 311 (62.2%)

Prediction Set Sizes (unselected only):
  Mean: 1.90, Median: 2


In [5]:
results['set_sizes']

array([105, 114, 118,  98, 157])

In [7]:
np.sum(aps_set != scp_res['prediction_sets'])

np.int64(0)

In [11]:
n = 1000
m = 500
K = 10
alpha = 0.3
N = 100

fdp = np.zeros((N, K))
eligs = np.zeros((N, K))
covs = []

for seed in range(N):
    np.random.seed(seed)
    cal_scores = np.random.dirichlet(np.ones(K), size=n)
    cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])   

    val_scores = np.random.dirichlet(np.ones(K), size=m) 
    val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)])  

    scp = StratifiedCP(eligibility="per_class", alpha_sel=0.3, alpha_cp=0.3, nonempty=False, rand=False)
    scp_res = scp.fit_predict(cal_scores, cal_labels, val_scores)

    covs.append(np.mean([scp_res['prediction_sets'][i,val_labels[scp_res['all_selected'][K][i]]] for i in range(len(scp_res['all_selected'][K]))]))

    for k in range(K):
        sel_k = (scp_res['all_selected'][k]).astype(int)
        eligs[seed, k] = np.sum(np.argmax(val_scores, axis=1)[sel_k] != k)
        fdp[seed, k] = np.mean(val_labels[sel_k] == k) if len(sel_k)>0 else 0.

print(np.mean(fdp, axis=0))
print(np.mean(eligs, axis=0)) 
print(np.mean(covs))


[0.19564756 0.13859912 0.14933397 0.22118323 0.16586239 0.24485725
 0.18413456 0.16980556 0.20045192 0.17636412]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.697185076509067


# Test utility-based

In [2]:
import stratcp as scp
import numpy as np

K = 5
n = 1000
m = 500

# Define similarity matrix between classes
# Higher values = more similar (range [0, 1])
similarity_matrix = np.array([
    [1.0, 0.9, 0.3, 0.3, 0.1],  # Class 0: very similar to 1
    [0.9, 1.0, 0.3, 0.3, 0.1],  # Class 1: very similar to 0
    [0.3, 0.3, 1.0, 0.9, 0.1],  # Class 2: very similar to 3
    [0.3, 0.3, 0.9, 1.0, 0.1],  # Class 3: very similar to 2
    [0.1, 0.1, 0.1, 0.1, 1.0],  # Class 4: dissimilar to all
])
 
cal_scores = np.random.dirichlet(np.ones(K), size=n)
cal_labels = np.array([np.random.choice(K, p=cal_scores[i,:]) for i in range(n)])   

val_scores = np.random.dirichlet(np.ones(K), size=m) 
val_labels = np.array([np.random.choice(K, p=val_scores[i]) for i in range(m)]) 

# Use standard StratifiedCP 
scp_standard = scp.StratifiedCP(
    score_fn='aps',
    similarity_matrix=similarity_matrix,
)
results_standard = scp_standard.fit_predict(cal_scores, cal_labels, val_scores, val_labels) 

avg_sim, overall_sim = scp.eval_similarity(
    results_standard['prediction_sets'],
    similarity_matrix
)
print(f"Standard CP average pairwise similarity: {overall_sim:.3f}")

# Use utility-aware CP
scp_utility = scp.StratifiedCP(
    score_fn='utility',
    similarity_matrix=similarity_matrix,
    utility_method='greedy',
    alpha_sel=0.1,
    alpha_cp=0.1
)
results = scp_utility.fit_predict(cal_scores, cal_labels, val_scores, val_labels)

# Evaluate prediction set coherence 
avg_sim, overall_sim = scp.eval_similarity(
    results['prediction_sets'],
    similarity_matrix
)
print(f"Utility-aware average pairwise similarity: {overall_sim:.3f}")


Standard CP average pairwise similarity: 0.345
Utility-aware average pairwise similarity: 0.442
